In [1]:
from loader_ref import ExcelLoader

In [2]:
from language_ref import Language

In [3]:
loader=ExcelLoader("/workspaces/de.uke.iam.automapping/src/automapping/VM_Fragebögen_HCHS_10000_20210708.xlsx", 'Kurzname', 'Langname')

In [4]:
#sample = loader.load(Language.GERMAN)[0]

In [4]:
samples = loader.load(Language.GERMAN)

In [6]:
from translator_ref import HuggingFace

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model_translator = HuggingFace(Language.GERMAN, Language.ENGLISH)

In [8]:
from preprocessor_ref import SpacyPreprocessor

In [9]:
model_spacy=SpacyPreprocessor(["lowercase", "stopwords", "punctuation", "lemmatization"])

In [10]:
for sample in samples:
    sample = model_translator.translate(sample)
    sample = model_spacy.transform(sample)

## Concepts

In [11]:
from concepts_ref import Concepts

In [12]:
import pandas as pd

In [13]:
concepts = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [14]:
synonyms = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT_SYNONYM.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [15]:
vocabulary = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/VOCABULARY.csv',on_bad_lines='skip', delimiter='\t')

In [16]:
concepts = Concepts.concatenate_concept_with_their_synonyms(concepts, synonyms, vocabulary, 'SNOMED')


/workspaces/de.uke.iam.automapping/src/automapping_ref/concepts_ref.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concepts["concept_name"] = (


In [17]:
from mapper_ref import TfIdf

In [18]:
model_tfidf = TfIdf(concepts)


In [19]:
from detection_ref import Predictions

In [20]:
df_all = pd.DataFrame()
for sample in samples:
    sample = model_tfidf(sample)
    predictions = Predictions(sample)
    df_result=Predictions.to_df(predictions, 5)
    df_all = pd.concat([df_all, df_result], ignore_index=True)


In [22]:
df_all

,SourceID,SourceName,targetConceptName,targetConceptID,targetConceptCode,targetDomainID,targetVocabularyVersion,MatchScore
0,E0,feeding meatless past 12 months,past,4132507,410513005,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.597550
1,E0,feeding meatless past 12 months,infant feeding at 4 months,4217242,417489001,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.510034
2,E0,feeding meatless past 12 months,infant feeding at 9 months,4014595,169990005,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.510034
3,E0,feeding meatless past 12 months,infant feeding at 6 months,4014725,169983000,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.510034
4,E0,feeding meatless past 12 months,infant feeding at 3 months,4016066,169977003,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.510034
5,E1,eat differently today 12 months ago,salmonella ago,4238832,57255003,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.402470
6,E1,eat differently today 12 months ago,symptom started months ago,4038056,162439003,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.397970
7,E1,eat differently today 12 months ago,does eat,4121069,288886005,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.347509
8,E1,eat differently today 12 months ago,hears differently in two ears,4082428,247283005,Synonym,2020-07-31 SNOMED CT International Edition; 20...,0.331143
9,E1,eat differently today 12 months ago,ability to eat,4124969,288883002,Observation,2020-07-31 SNOMED CT International Edition; 20...,0.330836
